# Evolving network architectures

By [Marcus Ghosh](https://profiles.imperial.ac.uk/m.ghosh/).

With inspiration from [NEAT](https://neat-python.readthedocs.io/en/latest/) and [Neuro4ML](https://neuro4ml.github.io/). 

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ghoshm/Evo_tutorial/Evo_tutorial.ipynb)

## Aim

We're going to write a simple evolutionary algorithm, and use it to solve a task. 

There are 3 parts to the exercise - which you should work on in pairs: 

> 0. Understand the network model. 
> 1. Write an evolutionary algorithm. 
> 2. A friendly contest. 

For the last 5 minutes we'll discuss what worked and how the tutorial could be improved! 

## Setup

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import os 

from src import *

# For Google Colab
if not os.path.exists('Data/test_current.csv'):
  !git clone https://github.com/ghoshm/Evo_tutorial.git 
  %cd /Evo_tutorial

## Task

Our goal is to approximate the [Hodgkin-Huxley model](https://en.wikipedia.org/wiki/Hodgkin%E2%80%93Huxley_model) - which describes how neurons respond to electrical inputs. 

Specifically, this model translates sequences of continuous values (representing time-varying input currents) into binary vectors (denoting if the model produced an output spike or not at each time step). 

In *Data* you'll find **train** and **test** sets of: 
 
* **Current** - inputs that were provided to the HH model. A numpy array of shape repeats x time which we'll call ```_I```. E.g. ```train_I```. 

* **Spikes** - contains two numpy vectors which we'll term: 
    * ```_spike_times``` - the time of each spike from the HH model (in ms).
    * ```_spike_idx``` - which repeat each spike comes from. 

In each data set there are: 
* **Train** - 100 repeats, with 10,000 time steps each. 
* **Test** - 50 repeats, with 10,000 time steps each.

We'll start by loading the training data. 

In the interest of time, we'll just focus on the first 200ms (feel free to use more or less data depending on how fast your machine is).

In [3]:
# Load training data
train_I = np.loadtxt('Data/train_current.csv') # shape (repeats, num_time_steps)
train_spike_times, train_spike_idx = np.loadtxt('Data/train_spikes.csv') # shape (num_spikes,)

# Crop 
time_limit = 200 # in ms
train_I = train_I[:, :time_limit*10]
train_spike_idx = train_spike_idx[train_spike_times < time_limit]
train_spike_times = train_spike_times[train_spike_times < time_limit]

To get a feel for the data try plotting some input currents and the model's output spikes. 

In [ ]:
n = np.random.randint(low=0, high=train_I.shape[0]) # sample a random repeat
HH_spikes = train_spike_times[train_spike_idx == n] # collect HH model spikes 

plt.plot(np.arange(len(train_I[n])) * 0.1, train_I[n], 'k', alpha=0.25, label="Input current")
plt.scatter(HH_spikes, np.ones(len(HH_spikes)), marker=".", color="xkcd:purple", label="HH model (spikes)")
plt.xlabel("Time (ms)")
plt.legend()

## 0. Network model

To approximate the HH model, we're going to try to evolve a neural network model.

The core of this model will be the ```RecurrentNetwork``` class in src.py. 

As a start, we'll initialise a model (an instance of the class) with: 
* 1 input unit - for the current.
* 1 hidden unit. 
* 2 output units.

When we pass inputs through the network (i.e. forward through the model), each output unit will acquire a value (or activation). To readout "spikes", we'll take an ```argmax``` of these units at each time step.      

Note that, for now, our model has no connections / weights.

In [ ]:
network = RecurrentNetwork(n_inputs=1, n_hidden=1, n_outputs=2)
network.connections 

We can use the ```plot_approximation``` and ```mean_vr_distance``` functions (from src.py) to test our model. 

With no connections, the model will just guess randomly (spike or no spike) at every time step and perform badly. 

Below we'll visualise and quantify this:
* **Plot** - in response to the same time-varying inputs, our network model (in green) will spike a lot more than the HH model (in purple). 

* **Metric** - for all of the train samples, we can measure the similarity between our network model's spikes and the HH model's spikes, using a metric known as the [van Rossum distance](http://www.scholarpedia.org/article/Measures_of_spike_train_synchrony). A lower value indicates a better match. For now, our model will have a high value. 

In [ ]:
plot_approximation(network, train_spike_times, train_spike_idx, train_I)
d = mean_vr_distance(network, train_spike_times, train_spike_idx, train_I, disable_reporting=False)

Using the ```.add_connections``` and ```.add_nodes``` functions (in ```RecurrentNetwork```) we can add connections and nodes to our network which may or may not improve it's performance.

Note: if your model spikes very little (or not at all) the score will be good (as the HH model's output is very sparse).  

In [ ]:
network.add_nodes(1)
network.add_connections(5)

print(network.connections)

plot_approximation(network, train_spike_times, train_spike_idx, train_I)
d = mean_vr_distance(network, train_spike_times, train_spike_idx, train_I, disable_reporting=False)

> Task 0. Have a look through the ```RecurrentNetwork``` class in src.py to get a feel for how the model is initialised, how to forward through it and how adding nodes and connections works. 

## 1. Writing an evolutionary algorithm 

To optimise our model, we're going to write a simple evolutionary algorithm in 4 steps:

* Generate a population of networks 
* Evaluate each network's fitness 
* Rank network's by their fitness 
* Make a new population by varying the best network(s)

Then, we'll wrap these steps into a function.

> Task 1: Fill in the code below to create a simple evolutionary algorithm.

In [8]:
# Generate a population of networks
population_size = 5 # the number of networks in each generation
networks = []
for n in range(population_size): 
    # Fill in code here
    pass 

In [9]:
# Evaluate each network's fitness 
fitness = []
for n in range(population_size):
    # Fill in code here 
    pass 

In [10]:
# Rank network's by their fitness 
# Fill in code here

In [11]:
# Make a new population by varying the best network(s)
# Fill in code here

In [12]:
# Now, wrap all of the above code into a function 

# I've suggested some inputs and outputs here, but you may want to add others! 

def evolve_networks(population_size, i_connections, i_nodes, epochs):
    """
    Evolves networks to solve a given task.  
    Arguments: 
        population_size: the number of networks per generation. 
        i_connections: nodes per network in generation 0. 
        i_nodes: connections per network in generation 1. 
        epochs: the number of generations to run. 
    Outputs: 
        networks: a list of networks from the final generation. 
    """
    # Generate a population of networks 
    networks = []
    for n in range(population_size):
        # Fill in code
        pass

    for _ in tqdm(range(epochs)):
        # Evaluate each network's fitness

        # Rank networks by their fitness

        # Make a new population by varying the best network(s)

        pass
    
    return networks

In [ ]:
# Finally, try testing your function with a small number of epochs

# Note: using tqdm as suggested above will allow you to estimate your codes run time
    # And save you waiting around.

networks = evolve_networks()

## 2. Contest

Now we have our evolutionary algorithm up and running, we'll have a friendly contest to see who can get the best (lowest) test score. 

To improve your score you could try anything you like! 

For example you could:
* Add new functions, to the RNN class, to vary networks (e.g. alter weights or even activation functions).
* Add a bias term to each unit, and functions to vary these. 
* Speed up the code by writing a more efficient forward pass (using matrix multiplication) - which will allow you to run more generations. 

Note: not spiking gets a good score, so may end up being a local minimum that is hard to escape. To avoid this, you could alter your fitness function to score models with few or no spikes as inf.

> Task 2: Try to evolve the best model by adding functionality to your code.  

In [21]:
# Load test data
test_I = np.loadtxt('Data/test_current.csv') # shape (repeats, num_time_steps)
test_spike_times, test_spike_idx = np.loadtxt('Data/test_spikes.csv') # shape (num_spikes,)

# Crop 
test_I = test_I[:, :time_limit*10]
test_spike_idx = test_spike_idx[test_spike_times < time_limit]
test_spike_times = test_spike_times[test_spike_times < time_limit]

In [ ]:
# Test your model
plot_approximation(network, test_spike_times, test_spike_idx, test_I)
d = mean_vr_distance(network, test_spike_times, test_spike_idx, test_I, disable_reporting=False)